In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import OneHotEncoder

In [ ]:
import psycopg2
import pandas as pd

# Parámetros de conexión
host = "localhost"        # o la IP de tu servidor
port = "5432"
dbname = "datos_historicos_componentes"
user = "postgres"
password = "postgres"

# Conexión a la base de datos
try:
    conn = psycopg2.connect(
        host=host,
        port=port,
        dbname=dbname,
        user=user,
        password=password
    )
    print("Conexión exitosa a la base de datos.")

    # Consulta a la tabla fallas_correctivas
    vida_util = pd.read_sql_query("SELECT * FROM vida_util_componentes;", conn)
    preventivo = pd.read_sql_query("SELECT * FROM mantenimiento_preventivo;", conn)
    fallas = pd.read_sql_query("SELECT * FROM fallas_correctivas;", conn)

    print(vida_util.head())
    print("----------------------")
    print(preventivo.head())
    print("----------------------")
    print(fallas.head())
    print("----------------------")

except Exception as e:
    print(f"Ocurrió un error: {e}")

finally:
    if 'conn' in locals() and conn:
        conn.close()
        print("Conexión cerrada.")


In [2]:
# Consulta a la tabla fallas_correctivas
# Forma correcta para XLSX:
vida_util = pd.read_excel("Datos_Historicos_Completos.xlsx", sheet_name=2)  # Hoja 3 (índice 2)
preventivo = pd.read_excel("Datos_Historicos_Completos.xlsx", sheet_name=0)  # Hoja 1
fallas = pd.read_excel("Datos_Historicos_Completos.xlsx", sheet_name=1)  # Hoja 2

In [ ]:
# 2. Agregaciones por componente
def features_from_preventivo(df):
    agg = df.groupby('id_componente').agg({
        'id_mantenimiento': 'count',
        'costo_usd': 'sum',
        'tiempo_inactividad_horas': 'sum',
        'tipo_mantenimiento': lambda x: x.mode()[0] if not x.mode().empty else 'Desconocido'
    }).rename(columns={
        'id_mantenimiento': 'num_mantenimientos',
        'costo_usd': 'costo_total_mant',
        'tiempo_inactividad_horas': 'tiempo_inact_mant',
        'tipo_mantenimiento': 'mant_frecuente'
    })
    return agg

def features_from_fallas(df):
    agg = df.groupby('id_componente').agg({
        'id_falla': 'count',
        'costo_reparacion_usd': 'sum',
        'tiempo_inactividad_horas': 'sum',
        'causa_falla': lambda x: x.mode()[0] if not x.mode().empty else 'Desconocida'
    }).rename(columns={
        'id_falla': 'num_fallas',
        'costo_reparacion_usd': 'costo_total_fallas',
        'tiempo_inactividad_horas': 'tiempo_inact_fallas',
        'causa_falla': 'causa_falla_frecuente'
    })
    return agg

feat_mant = features_from_preventivo(preventivo)
feat_fallas = features_from_fallas(fallas)

In [ ]:
# 3. Unir features con tabla principal
df = vida_util.set_index('id_componente')\
    .join(feat_mant, how='left')\
    .join(feat_fallas, how='left')

df.fillna({
    'num_mantenimientos': 0,
    'costo_total_mant': 0,
    'tiempo_inact_mant': 0,
    'mant_frecuente': 'Ninguno',
    'num_fallas': 0,
    'costo_total_fallas': 0,
    'tiempo_inact_fallas': 0,
    'causa_falla_frecuente': 'Ninguna'
}, inplace=True)

In [ ]:
# 4. Codificar variables categóricas
cat_cols = ['mant_frecuente', 'causa_falla_frecuente']
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded = encoder.fit_transform(df[cat_cols])
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(cat_cols), index=df.index)

In [ ]:
# 5. Preparar el dataset
X = pd.concat([df.drop(['componente', 'vida_util_estimada_anios'] + cat_cols, axis=1), encoded_df], axis=1)
y = df['vida_util_estimada_anios']

In [ ]:
# 6. Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# 7. Entrenar el modelo
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

In [ ]:
# 8. Evaluar
y_pred = model.predict(X_test)
print("MSE:", mean_squared_error(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("R2 Score:", r2_score(y_test, y_pred))

In [ ]:
# 9. Importancia de variables
importances = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)
print("\nImportancia de las variables:")
print(importances.head(10))

In [3]:
# Convertir fechas
preventivo['fecha_inspeccion'] = pd.to_datetime(preventivo['fecha_inspeccion'])
fallas['fecha_falla'] = pd.to_datetime(fallas['fecha_falla'])

# 2. Calcular vida útil real por componente
primer_mant = preventivo.groupby('id_componente')['fecha_inspeccion'].min()
ultima_falla = fallas.groupby('id_componente')['fecha_falla'].max()

vida_util_real = (ultima_falla - primer_mant).dt.days / 365.25
vida_util_real = vida_util_real.dropna().reset_index().rename(columns={0: 'vida_util_real_anios'})

# 3. Agregar features de mantenimiento y fallas
def features_from_preventivo(df):
    agg = df.groupby('id_componente').agg({
        'id_mantenimiento': 'count',
        'costo_usd': 'sum',
        'tiempo_inactividad_horas': 'sum',
        'tipo_mantenimiento': lambda x: x.mode()[0] if not x.mode().empty else 'Desconocido'
    }).rename(columns={
        'id_mantenimiento': 'num_mantenimientos',
        'costo_usd': 'costo_total_mant',
        'tiempo_inactividad_horas': 'tiempo_inact_mant',
        'tipo_mantenimiento': 'mant_frecuente'
    })
    return agg

def features_from_fallas(df):
    agg = df.groupby('id_componente').agg({
        'id_falla': 'count',
        'costo_reparacion_usd': 'sum',
        'tiempo_inactividad_horas': 'sum',
        'causa_falla': lambda x: x.mode()[0] if not x.mode().empty else 'Desconocida'
    }).rename(columns={
        'id_falla': 'num_fallas',
        'costo_reparacion_usd': 'costo_total_fallas',
        'tiempo_inactividad_horas': 'tiempo_inact_fallas',
        'causa_falla': 'causa_falla_frecuente'
    })
    return agg

feat_mant = features_from_preventivo(preventivo)
feat_fallas = features_from_fallas(fallas)

# 4. Construir dataset final
df = vida_util_real.set_index('id_componente')\
    .join(feat_mant, how='left')\
    .join(feat_fallas, how='left')

df.fillna({
    'num_mantenimientos': 0,
    'costo_total_mant': 0,
    'tiempo_inact_mant': 0,
    'mant_frecuente': 'Ninguno',
    'num_fallas': 0,
    'costo_total_fallas': 0,
    'tiempo_inact_fallas': 0,
    'causa_falla_frecuente': 'Ninguna'
}, inplace=True)

# 5. Codificar variables categóricas
cat_cols = ['mant_frecuente', 'causa_falla_frecuente']
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded = encoder.fit_transform(df[cat_cols])
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(cat_cols), index=df.index)

# 6. Preparar features y variable objetivo
X = pd.concat([df.drop(cat_cols + ['vida_util_real_anios'], axis=1), encoded_df], axis=1)
y = df['vida_util_real_anios']

# 7. Split y entrenamiento
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# 8. Evaluación del modelo
y_pred = model.predict(X_test)
print("MSE:", mean_squared_error(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("R2 Score:", r2_score(y_test, y_pred))

# 9. Importancia de variables
importances = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)
print("\nImportancia de las variables:")
print(importances.head(10))


MSE: 9.778369849160512
MAE: 2.8615103810175637
R2 Score: -2.130598304546924

Importancia de las variables:
costo_total_fallas                          0.255871
mant_frecuente_Lubricación                  0.128427
tiempo_inact_mant                           0.126905
tiempo_inact_fallas                         0.087163
causa_falla_frecuente_Sobrecalentamiento    0.081290
num_fallas                                  0.062705
causa_falla_frecuente_Corrosión             0.060717
num_mantenimientos                          0.058099
costo_total_mant                            0.047688
mant_frecuente_Revisión                     0.043860
dtype: float64


In [ ]:
# 1. Hacer predicciones para todos los componentes
y_pred_all = model.predict(X)

# 2. Crear DataFrame con los resultados
resultados = pd.DataFrame({
    'id_componente': df.index,
    'vida_util_real_anios': y,
    'vida_util_predicha_anios': y_pred_all
})

# 3. Agregar información adicional para contexto
resultados = resultados.join(df[['num_mantenimientos', 'num_fallas', 'mant_frecuente', 'causa_falla_frecuente']])

# 4. Calcular diferencia entre lo real y lo predicho
resultados['diferencia_anios'] = resultados['vida_util_real_anios'] - resultados['vida_util_predicha_anios']
resultados['error_porcentual'] = (resultados['diferencia_anios'] / resultados['vida_util_real_anios']) * 100

# 5. Ordenar por mayor diferencia absoluta para análisis
resultados = resultados.sort_values(by='diferencia_anios', key=abs, ascending=False)

# 6. Guardar a CSV
resultados.to_csv('comparacion_vida_util.csv', index=False, float_format='%.2f')

print("Archivo 'comparacion_vida_util.csv' generado exitosamente.")
print(f"Total de componentes analizados: {len(resultados)}")
print(f"Diferencia promedio: {resultados['diferencia_anios'].mean():.2f} años")